In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from data_extraction import data_extraction, data_downsample
from model import SRmodel

In [7]:
train_HR, train_LR = data_extraction(path='/content/drive/My Drive/Image_super_resolution/train_data/')
val_HR, val_LR = data_extraction('/content/drive/My Drive/Image_super_resolution/val_data/')

In [28]:
print(train_HR.shape, train_LR.shape)
print(val_HR.shape, val_LR.shape)

(2870, 256, 256, 3) (2870, 128, 128, 3)
(728, 256, 256, 3) (728, 128, 128, 3)


In [ ]:
model = SRmodel(ks=5, kr=3, filters=64, lr=0.02)

In [30]:
# aug = tf.keras.preprocessing.image.ImageDataGenerator(
#                         rotation_range=20,
# 	                      zoom_range=0.15,
# 	                      width_shift_range=0.2,
# 	                      height_shift_range=0.2,
# 	                      shear_range=0.15,
# 	                      horizontal_flip=True,
# 	                      fill_mode="nearest")

In [35]:
BASE = 'SRModel' 
BATCH_SIZE = 16
epochs = 100

In [32]:
import pandas as pd

def save_history(history):
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    base_file = '/content/drive/My Drive/Image_super_resolution/second_train/' + BASE + '.json' 
    with open(base_file, mode='w') as f:
        hist_df.to_json(f)

In [33]:
filepath = '/content/drive/My Drive/Image_super_resolution/second_train/' + BASE + '.hdf5'
saving = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                            monitor='val_loss',
                                            save_best_only=True,
                                            save_weights_only=True)

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
restore_best_weights=True)

# def exponential_decay_fn(epochs, lr):
#   return lr*0.1**(1/20)

# lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
history = model.fit(
    train_LR, train_HR
    epochs=epochs,
    batches_size=BATCH_SIZE,
    validation_data=(val_LR, val_HR),
    verbose=1,
    callbacks=[lr_scheduler, saving, early_stopping_cb]
    )   
    
save_history(history)

In [ ]:
num_ep = len(history.history['loss'])
plt.figure
plt.plot(np.arange(0, num_ep), history.history['loss'], label='train_loss')
plt.plot(np.arange(0, num_ep), history.history['val_loss'], label='val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc='upper right')

In [3]:
test_HR, test_LR = data_extraction('/content/drive/My Drive/Image_super_resolution/test_data/')

In [8]:
model.load_weights('/content/drive/My Drive/Image_super_resolution/first_train/SRModel.hdf5')
train_SR = model.predict(train_LR, batch_size=16)
val_SR = model.predict(val_LR, batch_size=16)
test_SR = model.predict(test_LR)

In [10]:
import keras.backend as K

train_psnr = tf.math.reduce_mean(tf.image.psnr(train_SR, train_HR, max_val=1.))
val_psnr = tf.math.reduce_mean(tf.image.psnr(val_SR, val_HR, max_val=1.))
test_psnr = tf.math.reduce_mean(tf.image.psnr(test_SR, test_HR, max_val=1.))
print('train_psnr: ', K.get_value(train_psnr))
print('val_psnr: ', K.get_value(val_psnr))
print('test_psnr: ', K.get_value(test_psnr))

train_psnr:  30.827452
val_psnr:  32.012836
test_psnr:  27.869999
